## Setup

In [ ]:
!pip3 install nltk emoji==0.6.0 -q
!pip3 install torchinfo -q
!pip3 install kagglehub -q
!pip3 install kaggle -q
!pip3 install transformers -q
!pip3 install evaluate -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cu

ERROR: Could not find a version that satisfies the requirement TweetNormalizer (from versions: none)
ERROR: No matching distribution found for TweetNormalizer


In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# list the elements in the current working directory
print("current directory: ", os.getcwd())
#os.chdir('content/drive/MyDrive/DATASCI 266/Project') # Use when working on Berkeley's Google Drive
os.chdir('drive/MyDrive/Project') # Use when working on Personal Google Drive
os.listdir()

current directory:  /content


['Project Proposal.gdoc',
 'TweetNormalizer.py',
 '__pycache__',
 'results_baseline',
 'wandb',
 'Crypto Sentiment Analysis_Data Preparation.ipynb',
 'Tweets_full_normalized.csv',
 'baseline_model',
 'bertweet_model',
 'small_train_dataset.csv',
 'small_eval_dataset.csv',
 'bertkeep_model',
 'Crypto Sentiment Analysis_BERTweet.ipynb',
 'Copy of Lora_(Roberta_Large)_pytorch.ipynb',
 'filtered_tweets.csv',
 'filtered_sample_2500.csv',
 'Copy of Crypto Sentiment Analysis.ipynb',
 'Normalized_filtered_tweets.csv',
 'small_train_dataset_filtered.csv',
 'small_eval_dataset_filtered.csv',
 'Copy of Crypto Sentiment Analysis_BERTweet.ipynb',
 'Crypto Sentiment Analysis.ipynb',
 'File Index.gsheet']

In [ ]:
import kagglehub
import pandas as pd
import numpy as np

import torch
from torchinfo import summary # to show the summary of the models
import evaluate # for metric evaluation during training from Huggingface
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Dataset # from Huggingface

## Data Analysis

Data Source: Kaggle  
Location in Kaggle: fabioturazzi/cryptocurrency-tweets-with-sentiment-analysis  
File name: tweets_sentiment.csv    
Link: https://www.kaggle.com/api/v1/datasets/download/fabioturazzi/cryptocurrency-tweets-with-sentiment-analysis?dataset_version_number=1  

In [ ]:
# Download latest version
# Download latest version
path = kagglehub.dataset_download("fabioturazzi/cryptocurrency-tweets-with-sentiment-analysis")

print("Path to dataset files:", path)

100%|██████████| 178M/178M [00:01<00:00, 143MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/fabioturazzi/cryptocurrency-tweets-with-sentiment-analysis/versions/1


In [ ]:
file_name = "tweets_sentiment.csv"
df = pd.read_csv(path + "/" + file_name)
df.head()

<ipython-input-6-758bb0830731>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + "/" + file_name)


,Unnamed: 0,id,conversation_id,created_at,date,time,timezone,user_id,username,name,...,retweet_date,translate,trans_src,trans_dest,search,decoded_tweet,neg,neu,pos,compound
0,0,3.177290e+17,3.177290e+17,2013-03-29 13:05:03 Hor. de Verão do Pacífico ...,2013-03-29,13:05:03,-800,14335498.0,newsycombinator,Hacker News,...,NaN,NaN,NaN,NaN,cryptocurrency,Bitcoin: The Cyberpunk Cryptocurrency http://...,0.0,1.000,0.000,0.000
1,1,3.238530e+17,3.238530e+17,2013-04-15 10:37:11 Hor. de Verão do Pacífico ...,2013-04-15,10:37:11,-800,972651.0,mashable,Mashable,...,NaN,NaN,NaN,NaN,cryptocurrency,Bitcoin Isn't the Only Cryptocurrency in Town ...,0.0,1.000,0.000,0.000
2,2,3.236100e+17,3.236100e+17,2013-04-14 18:34:04 Hor. de Verão do Pacífico ...,2013-04-14,18:34:04,-800,5988062.0,theeconomist,The Economist,...,NaN,NaN,NaN,NaN,cryptocurrency,"#Bitcoin, a ""cryptocurrency"", went on a tear l...",0.0,1.000,0.000,0.000
3,3,3.249920e+17,3.249920e+17,2013-04-18 14:04:24 Hor. de Verão do Pacífico ...,2013-04-18,14:04:24,-800,14248784.0,drewtoothpaste,drewtoothpaste,...,NaN,NaN,NaN,NaN,cryptocurrency,I'm going to make my OWN crypto-currency and e...,0.0,0.885,0.115,0.381
4,4,3.326200e+17,3.326200e+17,2013-05-09 15:14:19 Hor. de Verão do Pacífico ...,2013-05-09,15:14:19,-800,2100521.0,jonathancoulton,Jonathan Coulton,...,NaN,NaN,NaN,NaN,cryptocurrency,"Your momma's cryptocurrency is so virtual, she...",0.0,1.000,0.000,0.000


In [ ]:
df2 = df[['date','tweet','neg','neu','pos','compound']].copy()

In [ ]:
df2.dropna(inplace=True)
df2.drop_duplicates(inplace=True)

In [ ]:
print(df2.shape)
df2.info()

(563799, 6)
<class 'pandas.core.frame.DataFrame'>
Index: 563799 entries, 0 to 824907
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   date      563799 non-null  object 
 1   tweet     563799 non-null  object 
 2   neg       563799 non-null  float64
 3   neu       563799 non-null  float64
 4   pos       563799 non-null  float64
 5   compound  563799 non-null  float64
dtypes: float64(4), object(2)
memory usage: 30.1+ MB


In [ ]:
df2['tweet_clean'] = df2['tweet'].str.replace(r'@\w+', '', regex=True)
df2['tweet_clean'] = df2['tweet_clean'].str.replace(r'http\S+|www.\S+', '', regex=True)
# Keep the word, drop the #
df2['tweet_clean'] = df2['tweet_clean'].str.replace(r'#', '', regex=True)
df2['tweet_clean'] = df2['tweet_clean'].str.replace(r'\s+', ' ', regex=True).str.strip()




In [ ]:

# Step 1: Add the 'label' column to df2 with a default value of 0 (neutral)
df2['label'] = 0

# Step 2: Apply labels (positive/negative) only to the rows in df2_filtered
df2.loc[df2_filtered.index, 'label'] = df2_filtered['compound'].apply(lambda x: 2 if x > 0 else 1)


In [ ]:
df2['label'].value_counts()

,count
label,
0,352018
2,157764
1,54017


In [ ]:
#df_filtered['label'] = df_filtered['compound'].apply(lambda x: 2 if x > 0 else 1)


In [ ]:

directory = '/content/drive/.shortcut-targets-by-id/1xT6cdCXD4s5pURHY0Jd8BkLnI-Qn4Hyy/Project'

df2.to_csv(f"{directory}/labelled_dataset.csv", index=False)


In [ ]:
import re

# Function to count the number of emojis in a tweet
def count_emojis(tweet):
    # Regular expression to match emojis
    emoji_pattern = re.compile("[\U0001F600-\U0001F64F"
                                "\U0001F300-\U0001F5FF"
                                "\U0001F680-\U0001F6FF"
                                "\U0001F700-\U0001F77F"
                                "\U0001F780-\U0001F7FF"
                                "\U0001F800-\U0001F8FF"
                                "\U0001F900-\U0001F9FF"
                                "\U0001FA00-\U0001FA6F"
                                "\U0001FA70-\U0001FAFF"
                                "\U00002702-\U000027B0"
                                "\U000024C2-\U0001F251]", flags=re.UNICODE)
    return len(re.findall(emoji_pattern, tweet))

# Updated filter function to include emoji count
def is_meaningful_sentiment(tweet, compound):
    if abs(compound) <= 0.2:
        return False

    # Keywords and joke markers
    keywords = [
        'bitcoin', 'btc', 'crypto', 'ethereum', 'eth',
        'dogecoin', 'blockchain', 'altcoin', 'token',
        'mining', 'wallet', 'exchange', 'defi', 'nft']
    joke_markers = ['your momma', 'lmao', 'rofl', 'amaze', 'wow', 'so much', "dad", 'bro', 'lol', 'Kanye', 'be a part of', 'read more at']

    # Filter based on keywords
    if not any(word in tweet.lower() for word in keywords):
        return False

    # Filter based on joke markers
    if any(joke in tweet.lower() for joke in joke_markers):
        return False

    # Filter based on emoji count (remove if more than 4 emojis)
    if count_emojis(tweet) > 4:
        return False

    return True

# Apply the filter to the dataframe
df2_filtered = df2[df2.apply(lambda row: is_meaningful_sentiment(row['tweet_clean'], row['compound']), axis=1)]


In [ ]:
# Step 1: Filter df2 using the same criteria
df2_filtered = df2[df2.apply(lambda row: is_meaningful_sentiment(row['tweet_clean'], row['compound']), axis=1)]

# Step 2: Add the 'label' column to df2 with a default value of 0 (neutral)
df2['label'] = 0

# Step 3: Apply labels (positive/negative) only to the rows in df2_filtered
df2.loc[df2_filtered.index, 'label'] = df2_filtered['compound'].apply(lambda x: 2 if x > 0 else 1)

In [ ]:
df2_filtered.shape

(308956, 7)

In [ ]:
pd.set_option('display.max_colwidth', None)
df2_filtered[['tweet_clean', 'neg', 'neu', 'pos', 'compound']].sample(10, random_state=200)

,tweet_clean,neg,neu,pos,compound
588861,"Over the past several years, bitcoin synthetics have swollen to nearly 5% of floating (read: non-lost) BTC. + Grayscale: 450k+ BTC + WBTC: 100k+ BTC + Coinshares: 65k+ BTC + renBTC/tBTC/sBTC: 25k+ BTC ETPs + Wrapped Assets could be 25%+ of network assets in a few more years.",0.000,0.931,0.069,0.3400
686182,If you're a bitcoiner who's been mostly ignoring ethereum but consider yourself open minded... and you have five mins.... You should try Argent,0.074,0.926,0.000,-0.2144
484048,"""There are more Mexicans that have used bitcoin than have ever engaged with the Mexican Stock Exchange."" shares how he started the crytocurrency LIVE at :",0.000,0.851,0.149,0.5994
710822,"Though defi is the new eth narrative, there’s been: - A recent uptick in NFT growth - Google introduced Stadia for gaming (one part of NFT gaming’s thesis) - Proving digital ownership has become one of Twilio’s priorities Here’s what’s actually going on in NFT-land 👇",0.000,0.949,0.051,0.3818
662861,"If you search for a successful real-world use of blockchain, take a look at Global Miles.",0.000,0.781,0.219,0.8074
579136,"I finally discovered an Australian exchange that supports a fresh address per deposit, and (bonus) native segwit incoming as well as outgoing. No alt bait-and-switch either. (I've used them successfully, not been paid in any way to endorse).",0.042,0.612,0.346,0.9313
116371,"CryptoRevolution has covered some Energi updates in his newest video! He talks about Energi 3.0 Public Testing, Energi Defense, and the work that is doing. Watch here: $NRG",0.000,0.947,0.053,0.2003
296502,â€ŒNovogratz predicts cryptocurrency market cap might touch $800 billion in 12 months I think Mike has this wrong and itâ€™ll be double this amount. $btc,0.127,0.873,0.000,-0.5610
119569,"Is cryptocurrency the future of money? On the latest episode of “Futurewatch”, supported by , we talk to about Bitcoin and whether digital coins can offer a viable alternative to existing currencies",0.000,0.937,0.063,0.3182
312068,Crypto winter is claiming its first victims as other projects begin to freeze up. This isn't the case for who will continue to . has lots of partnerships &amp; developments yet to be announced. y'all! â›„ï¸â„ï¸,0.057,0.917,0.026,-0.3382


In [ ]:

directory = '/content/drive/.shortcut-targets-by-id/1xT6cdCXD4s5pURHY0Jd8BkLnI-Qn4Hyy/Project'
print("Directory exists:", os.path.isdir(directory))


Directory exists: True


In [ ]:
csv_path = '/content/drive/.shortcut-targets-by-id/1xT6cdCXD4s5pURHY0Jd8BkLnI-Qn4Hyy/Project'

df2_filtered.to_csv(f"{directory}/labelled_dataset.csv", index=False)


# Widget for Looking at Tweets to assess accuracy of filter.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Sample data for demo (you can replace this with your actual df2_filtered)
tweets = df2_filtered[['tweet', 'compound']].sample(10, random_state=64).reset_index(drop=True)

# Create a display function for the tweet card
def display_tweet(index):
    tweet = tweets.iloc[index]
    tweet_text = tweet['tweet']
    compound_score = tweet['compound']

    # HTML widget for scrollable container with better contrast
    tweet_window = widgets.HTML(
        value=f'<div style="width: 100%; height: 150px; overflow-y: auto; padding: 15px; border: 1px solid #333; border-radius: 5px; background-color: #fff; color: #333; font-size: 16px; line-height: 1.5;">{tweet_text}</div>'
    )

    # Display tweet and compound score
    print(f"Compound Score: {compound_score}\n")
    display(tweet_window)

# Initialize index
current_index = 0

# Next button function
def next_tweet(b):
    global current_index
    current_index += 1
    if current_index >= len(tweets):
        current_index = 0  # loop back to the first tweet
    display_tweet(current_index)

# Previous button function
def prev_tweet(b):
    global current_index
    current_index -= 1
    if current_index < 0:
        current_index = len(tweets) - 1  # loop back to the last tweet
    display_tweet(current_index)

# Create next and previous buttons
next_button = widgets.Button(description="Next")
prev_button = widgets.Button(description="Previous")

# Bind buttons to functions
next_button.on_click(next_tweet)
prev_button.on_click(prev_tweet)

# Display the first tweet and buttons
display_tweet(current_index)
display(widgets.HBox([prev_button, next_button]))
